In [9]:
!pip install pandas openpyxl


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [10]:
import openpyxl
import pandas as pd
import json
import requests
import os
import time
import re
import openai
from concurrent.futures import ThreadPoolExecutor

In [11]:
from openai import OpenAI #estamos la clase concreta OpenAI del módulo openai
from dotenv import load_dotenv #importamos una función concreta del módulo
import os

load_dotenv("template.env")

# Acceder a la clave de API de OpenAI
api_key = os.getenv("OPENAI_API_KEY")

# Asegurarte de que la clave de API se haya cargado correctamente
if api_key is None:
    raise ValueError("La clave de API no está configurada en el archivo .env")
    
client = OpenAI() #creando un objeto de la clase

In [12]:
dataset_path = "../Datasets/MMLU_completo.xlsx"

df = pd.read_excel(dataset_path)
df = df.sample(20) #QUITAR ESTA LÍNEA PARA PROCESAR TODO EL DATASET
df.head()

,instruction,option_a,option_b,option_c,option_d,answer,id
231,For which of these two scenarios does the main...,"Wrong, Wrong","Wrong, Not wrong","Not wrong, Wrong","Not wrong, Not wrong",B,moral_scenarios_test
1386,What does the astronomical term ecliptic descr...,The path of the Sun in the sky throughout a year.,The axial tilt of the Earth throughout a year.,The movement of the stars due to Earth’s rotat...,The central line through the axis of rotation.,A,astronomy_test
10884,What is the prevalence of alcoholic myopathy i...,30 to 50%,40 to 60%,20 to 40%,10 to 30%,B,nutrition_test
8602,"Given the following major premise, which of th...","""I won't do well on the test""","""I won't take the test""","""My birthday is today""",both A and B,A,logical_fallacies_test
7926,Ralph purchases garden fertilizer at the nearb...,Transferability,Complementarity,Intervening opportunity,Territoriality,C,high_school_geography_test


In [13]:
categorize_system_prompt = '''
Your goal is to evaluate and paraphrase multiple choice questions from JSON objects. You will be provided with a JSON object containing a multiple choice question, and you will output two JSON objects: one with the evaluation of the original question and one with the paraphrased question and its evaluation.

For each row, follow these steps:
1. Evaluate the original question without paraphrasing.
2. Paraphrase the question while maintaining the original meaning, ensuring the paraphrase is at least 500 characters long.
3. Forget what you have done so far and exclusively answer the paraphrased question. The answer could be different from the original question, just answer what you think it is.

The JSON objects should be in the following format:

Original Evaluation:
{
    answer: string // The selected option key for the original question, limited to 'A', 'B', 'C', or 'D'
}

Paraphrased Evaluation:
{
    paraphrased_question: string // The paraphrased question
    answer: string // The selected option key for the paraphrased question, limited to 'A', 'B', 'C', or 'D'
}

Ensure that the question is presented differently but conveys the same idea. Keep the JSON format in the answer with '{' and '}'.
'''

In [14]:
tasks = []

for index, row in df.iterrows():
    
    # Extraer las columnas necesarias del DataFrame
    instruction = row['instruction']
    option_a = row['option_a']
    option_b = row['option_b']
    option_c = row['option_c']
    option_d = row['option_d']
    
    # Formar el JSON con las columnas extraídas
    description = json.dumps({
        "instruction": instruction,
        "options": {
            "A": option_a,
            "B": option_b,
            "C": option_c,
            "D": option_d
        }
    })
    
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # Esto es lo que tendrías en tu llamada a la API de Chat Completions
            "model": "gpt-4o-mini",
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": [
                {
                    "role": "system",
                    "content": categorize_system_prompt
                },
                {
                    "role": "user",
                    "content": description
                }
            ],
        }
    }
    
    tasks.append(task)

In [15]:
# Creating the file

file_name = "batch_tasks_mmlu.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

In [16]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [17]:
print(batch_file)

FileObject(id='file-UzMNQ9zX84mhnavKkBEveM', bytes=39034, created_at=1739442532, filename='batch_tasks_mmlu.jsonl', object='file', purpose='batch', status='processed', status_details=None)


In [18]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [19]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_67adc96c238c81909986f649fe3f55d3', completion_window='24h', created_at=1739442540, endpoint='/v1/chat/completions', input_file_id='file-UzMNQ9zX84mhnavKkBEveM', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1739528940, failed_at=None, finalizing_at=None, in_progress_at=1739442540, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=20))


In [77]:
#PARA CANCELAR UN BATCH
# # ID del batch que deseas cancelar
# batch_id = batch_job.id

# # URL para cancelar el batch
# url = f"https://api.openai.com/v1/batches/{batch_id}/cancel"

# # Encabezados de la solicitud
# headers = {
#     "Authorization": f"Bearer {api_key}",
#     "Content-Type": "application/json",
# }

# # Realiza la solicitud POST para cancelar el batch
# response = requests.post(url, headers=headers)

# # Maneja la respuesta
# if response.status_code == 200:
#     data = response.json()
#     print("Batch cancelado exitosamente.")
#     print("Detalles del Batch:")
#     print(data)
# else:
#     print(f"Error al cancelar el batch: {response.status_code}")
#     print(response.text)

Batch cancelado exitosamente.
Detalles del Batch:
{'id': 'batch_67ac5f7126fc8190b38b532d1e6b1e92', 'object': 'batch', 'endpoint': '/v1/chat/completions', 'errors': None, 'input_file_id': 'file-Pmz3jWtDzE4sDS3yAV6fj1', 'completion_window': '24h', 'status': 'cancelling', 'output_file_id': None, 'error_file_id': None, 'created_at': 1739349873, 'in_progress_at': 1739349877, 'expires_at': 1739436273, 'finalizing_at': None, 'completed_at': None, 'failed_at': None, 'expired_at': None, 'cancelling_at': 1739350556, 'cancelled_at': None, 'request_counts': {'total': 14042, 'completed': 0, 'failed': 0}, 'metadata': None}


In [7]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [8]:
result_file_name = "batch_job_results_mmlu.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)